In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path
import numpy as np
import pandas as pd
from pathlib import Path
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.dataSession import DataSession
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec

# For specific load functions:
from neuropy.core import DataWriter, NeuronType, Neurons, BinnedSpiketrain, Mua, ProbeGroup, Position, Epoch, Signal, Laps, FlattenedSpiketrains, Shank, Probe, ProbeGroup
from neuropy.io import OptitrackIO, PhyIO
from neuropy.utils.mixins.print_helpers import ProgressMessagePrinter, SimplePrintable, OrderedMeta

from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.utils.result_context import IdentifyingContext


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/home/halechr/cloud/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"


## Rachel:
active_data_mode_name = 'rachel'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('Rachel')

# basedir: Path = Path(r'W:\Data\Rachel\20230614_Rachel').resolve()

# basedir: Path = Path('/home/halechr/FastData/Rachel/20230614_Rachel/merged_20230614_2crs.GUI').resolve()

basedir: Path = Path('/home/halechr/FastData/Rachel/20230614_Rachel').resolve()
assert basedir.exists()

# filename: str = '20230614_Rachel'
# filename: str = '20230614_Rachel_2'
filename: str = 'merged_20230614_2'

Automatic pdb calling has been turned OFF


In [2]:

# RachelDataSessionFormat.initialize_data_directory(basedir)
## Builds the .neurons.npy:
# folder = Path('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/merged_M1_20211123_raw_phy')
# folder = Path(r'W:\Data\Rachel\20230614_Rachel')
folder = basedir.resolve()
phydata = PhyIO(folder)
# /home/halechr/FastData/Rachel/20230614_Rachel/params.py


# neuronIDs = pd.read_csv(r'W:\Data\Rachel\20230614_Rachel\cluster_q.tsv');

neuronIDs = pd.read_csv(basedir.joinpath('cluster_q.tsv'))
neurons = Neurons(spiketrains=phydata.spiketrains, t_stop=2*3600, sampling_rate=30000, neuron_ids = {1:'pyr1',2:'pyr2',3:'pyr3',4:'int1',5:'int2',6:'int3',7:"mua1",8:'mua2',9:'mua3'})
neurons.filename = folder.joinpath(f'{filename}.neurons.npy')
neurons.save()

# Probe Groups file
# TODO: Probe group generation
# shanks = []
# # channel_groups = sess.recinfo.channel_groups
# for i in range(8):
#     shank = Shank.auto_generate(
#         columns=1,
#         contacts_per_column=128,
#         xpitch=90,
#         ypitch=0,
#         y_shift_per_column=[0, 0],
#         channel_id=np.arange(0,128,1)
#         ),
	
# elec_IDs = np.arange(0,128,1)
# shanks = Shank.auto_generate(channels=1, contacts_per_column = 128)
# shanks = pd.read_csv('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/Probe.csv',delimiter=',',usecols=["ShankNumber"])
# prb = Probe(shanks)
# prbgroup = ProbeGroup()
# prbgroup.add_probe(prb)


## Builds the .position.npy:
# opti_folder = Path(r'W:\Data\Rachel\20230614_Rachel')
# opti_folder = basedir.resolve()
# opti_data = OptitrackIO(opti_folder)
# brelative = pd.read_csv(r'W:\Data\Rachel\20230614_Rachel\merged_M1_20211123_raw_behavior_relativetoLFP.csv',header = None)

csv_path = basedir.joinpath(f'20230614_positionData.csv')
# brelative = pd.read_csv(csv_path, header = None)
brelative = pd.read_csv(csv_path)
brelative
# Change column type to timedelta64[ns] for column: 'AbsoluteTime'
brelative = brelative.astype({'AbsoluteTime': 'timedelta64[ns]'})

brelative_seconds = brelative.AbsoluteTime.dt.total_seconds().to_numpy()
start_t = np.min(brelative_seconds)
# start_t = np.min(brelative.AbsoluteTime.to_numpy())
print(f'start_t: {start_t}')

t_relative = brelative_seconds - start_t
t_relative

print(f'brelative.shape: {brelative.shape}')
# d = {'t':brelative[0],'x':opti_data.z,'y':opti_data.x} 
d = {'t':brelative_seconds,'x':brelative.Z.to_numpy(),'y':brelative.X.to_numpy()} 

behaviordf = pd.DataFrame(data=d)
print(f'behaviordf.shape: {behaviordf.shape}')
position = Position(behaviordf)
# position.filename = Path(f'W:\Data\Rachel\20230614_Rachel\{filename}.position.npy')
position.filename = basedir.joinpath(f'{filename}.position.npy')
position.save()

## Builds the .paradigm.npy file from scratch:
# starts = [0, 5*60]
# stops = [5*60-1, 3.8398632e+03]
# labels = ['pre','maze']

paradigm_df = pd.DataFrame(dict(label=['Pre','Maze','Post'],
	start = ['11:15:49.000','12:02:19.095','13:08:37.999'],
	stops = ['11:53:19.384','13:04:54.815','14:53:22.047'],
))
# Change column type to timedelta64[ns] for columns: 'Starts', 'Stops'
paradigm_df = paradigm_df.astype({'start': 'timedelta64[ns]', 'stops': 'timedelta64[ns]'})


## Build and save the paradigm.npy
d = {'start':paradigm_df.start.dt.total_seconds().to_numpy(),
	 'stop':paradigm_df.stops.dt.total_seconds().to_numpy(),
	 'label':paradigm_df.label.to_list()} 
paradigmdf = pd.DataFrame(data=d)
paradigm = Epoch(paradigmdf)
# paradigm.filename = Path('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/merged_M1_20211123_raw_phy/merged_M1_20211123_raw.paradigm.npy')
paradigm.filename = basedir.joinpath(f'{filename}.paradigm.npy')
paradigm.save()

id column does not exist in cluster_info.tsv. Using cluster_id column instead.
merged_20230614_2.neurons.npy saved
start_t: 43339.095
brelative.shape: (221455, 4)
behaviordf.shape: (221455, 3)
merged_20230614_2.position.npy saved
merged_20230614_2.paradigm.npy saved


In [3]:

_test_session = RachelDataSessionFormat.build_session(basedir)


RequiredFileError: Required File: /home/halechr/FastData/Rachel/20230614_Rachel/merged_20230614_2.probegroup.npy does not exist.

In [ ]:
_test_session, loaded_file_record_list = RachelDataSessionFormat.load_session(_test_session)
_test_session
